In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20, 10

from datetime import date
from datetime import timedelta

from yahoofinancials import YahooFinancials

from scipy.stats import norm

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation

import warnings
warnings.filterwarnings('ignore')

In [2]:
end = '2021-07-28'
start = '2016-07-20'

In [3]:
# Change the loctaion directory

weight_df = pd.read_csv('../data/WeightsDF.csv')
weight_df.set_index('Portfolio', inplace=True)

In [4]:
stocks = np.array(weight_df.columns).tolist()

i1 = 0
i2 = 1
portfolio1 = []
portfolio2 = []
for stock in stocks:
    if weight_df[stock][i1] > 0:
        portfolio1.append(stock)
    if weight_df[stock][i2] > 0:
        portfolio2.append(stock)

In [5]:
portfolio_ticker = []
for stock in stocks:
    portfolio_ticker.append(stock+'.NS')

In [6]:
stocks += ['NIFTY']
portfolio_ticker += ['^NSEI']

In [7]:
for stock_name, stock_ticker in zip(stocks, portfolio_ticker):
    
    yf = YahooFinancials(stock_ticker)
    data = yf.get_historical_price_data(start, end, 'daily')
    globals()[stock_name] = pd.DataFrame(data[stock_ticker]['prices'])
    globals()[stock_name] = globals()[stock_name].drop('date', axis=1).set_index('formatted_date')

    globals()[stock_name]['Date'] = pd.to_datetime(globals()[stock_name].index, format="%Y-%m-%d")
    globals()[stock_name].set_index('Date', drop=False, inplace=True)

    globals()[stock_name] = globals()[stock_name].dropna()

In [8]:
closing_df = pd.DataFrame(index=TCS.index, columns=stocks)
for stock in stocks:
    closing_df[stock] = globals()[stock]['close']

In [9]:
# Change the loctaion directory

lstm_model = load_model('../data/lstm.h5')

In [10]:
scaler = MinMaxScaler(feature_range=(0,1))

In [11]:
look_back = 60
num_prediction = 90

In [12]:
def predict(num_prediction, model, close_data):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = end
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    
    return prediction_dates

In [13]:
pred_dates = predict_dates(num_prediction)

pred = pd.DataFrame(index=pred_dates, columns=stocks)

In [14]:
for stock in stocks:
    
    close_data = scaler.fit_transform(closing_df[stock].values.reshape((-1,1)))
    close_data = close_data.reshape((-1))
    
    forecast = predict(num_prediction, lstm_model, close_data)
    forecast = scaler.inverse_transform(forecast.reshape(1, -1))
    forecast = forecast.reshape((-1))
    
    pred[stock] = forecast

In [15]:
pred

,IRCTC,NESTLEIND,INDIAMART,TITAN,INFY,TATACONSUM,TCS,DIXON,HINDUNILVR,ATGL,DIVISLAB,POLYCAB,NIFTY
2021-07-28,2295.550049,18226.099609,7179.950195,1729.050049,1602.550049,769.750000,3182.949951,4518.100098,2342.399902,931.049988,4791.399902,1802.500000,15746.450195
2021-07-29,2279.042343,18199.247019,7268.504813,1735.103650,1607.761746,778.428017,3218.151959,4548.170917,2364.034711,915.976342,4826.568651,1854.652726,15860.218242
2021-07-30,2267.822716,18208.650799,7317.088697,1744.635816,1611.169311,779.758146,3215.495923,4552.890160,2362.575837,939.093920,4807.121839,1833.346782,15878.422769
2021-07-31,2267.337751,18197.417883,7344.221443,1749.480150,1610.274773,780.477759,3220.868552,4567.808458,2372.392607,953.736194,4791.245405,1835.288981,15901.173471
2021-08-01,2273.066697,18181.374505,7360.116750,1751.497395,1607.915080,780.995417,3229.799323,4586.381407,2386.158632,962.091425,4782.619795,1848.873006,15925.482679
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-22,2525.929676,19914.319071,8208.469715,1951.346558,1766.303689,874.367583,3563.568346,5214.758028,2646.109988,1065.158502,5394.634398,2092.938110,17124.822816
2021-10-23,2528.924032,19937.396932,8220.048917,1954.021993,1768.448549,875.589167,3567.677334,5222.578619,2649.151076,1066.620080,5402.516929,2095.494107,17140.149188
2021-10-24,2531.922708,19960.504158,8231.649254,1956.700956,1770.595774,876.811855,3571.791600,5230.407867,2652.195870,1068.085296,5410.410416,2098.053453,17155.491418
2021-10-25,2534.925294,19983.640749,8243.273302,1959.382740,1772.745226,878.036040,3575.911977,5238.245772,2655.245400,1069.554244,5418.312248,2100.617029,17170.847522


In [16]:
monthly_return = pred.resample('M').ffill().pct_change()
monthly_return

,IRCTC,NESTLEIND,INDIAMART,TITAN,INFY,TATACONSUM,TCS,DIXON,HINDUNILVR,ATGL,DIVISLAB,POLYCAB,NIFTY
2021-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-31,0.047935,0.030743,0.039774,0.038588,0.029826,0.041432,0.042498,0.055498,0.051197,0.042143,0.043274,0.070727,0.028413
2021-09-30,0.035816,0.035050,0.042289,0.042013,0.037048,0.043086,0.034794,0.046398,0.034647,0.040269,0.045041,0.036909,0.026849
2021-10-31,0.031210,0.030519,0.037146,0.036307,0.032073,0.037066,0.030352,0.039858,0.030246,0.035856,0.038766,0.032175,0.023474


In [17]:
monthly_closing_df = pred.resample('M').mean()
monthly_closing_df

,IRCTC,NESTLEIND,INDIAMART,TITAN,INFY,TATACONSUM,TCS,DIXON,HINDUNILVR,ATGL,DIVISLAB,POLYCAB,NIFTY
2021-07-31,2277.438215,18207.853828,7277.441287,1739.567416,1607.938970,777.103480,3209.366596,4546.742408,2360.350764,934.964111,4804.083949,1831.447122,15846.566169
2021-08-31,2334.015690,18441.894427,7486.674961,1780.584539,1628.774146,795.902043,3300.840091,4712.632788,2451.281398,978.856199,4889.101264,1927.596607,16141.400261
2021-09-30,2419.733228,19094.200801,7801.569369,1856.131075,1689.840061,830.788674,3417.653257,4936.087696,2538.145745,1014.393643,5114.222685,2002.299376,16578.279743
2021-10-31,2500.765789,19720.226839,8111.545630,1928.830839,1748.239291,864.075845,3529.021274,5148.899572,2620.544919,1052.992298,5328.303169,2071.460601,16995.728522


In [18]:
nifty_std = monthly_return['NIFTY'].std()

mstd = monthly_return.copy()
mstdl = mstd.corr(method='pearson')

In [19]:
monthly_stock_beta = pd.Series(index=stocks)

for stock in stocks:
    monthly_stock_beta[stock] = (mstdl['NIFTY'][stock] * mstd[stock].std()) / nifty_std

In [20]:
monthly_stock_beta

IRCTC         3.063382
NESTLEIND     0.252615
INDIAMART     0.690555
TITAN         0.658241
INFY         -0.146754
TATACONSUM    1.027902
TCS           2.276726
DIXON         2.970431
HINDUNILVR    3.772877
ATGL          1.278574
DIVISLAB      1.063843
POLYCAB       6.783032
NIFTY         1.000000
dtype: float64

In [21]:
w1 = weight_df.iloc[0].dropna()
w2 = weight_df.iloc[1].dropna()

In [22]:
metrics = ['Return','Volatility','Sharpe Ratio', 'VaR', 'Portfolio Beta', 'Treynor Ratio', 'Jensen Alpha']
metrics_df = pd.DataFrame(columns=metrics)

In [23]:
nifty_50_daily = pred['NIFTY'].pct_change().dropna()

In [24]:
risk_free_rate = 0.0602
alpha = 0.01
days = 21
market_return = nifty_50_daily.mean() * days 

In [25]:
# P1

r1 = np.zeros(len(metrics))
sb1 = monthly_stock_beta[portfolio1].copy()
pb1 = 0
tdf1 = monthly_return[portfolio1].copy()
twl1 = w1
tcovdf1 = tdf1.cov()

pr1 = np.sum(tdf1.mean() * twl1) * days
pstd1 = np.sqrt(np.dot(twl1.T,np.dot(tcovdf1, twl1))) * np.sqrt(days)

r1[0] = pr1
r1[1] = pstd1

# SHARPE RATIO => (portfolio_return - risk_free_rate) / volatility
r1[2] = (r1[0] - risk_free_rate) / r1[1]

# VaR => return - (Z * volatility)
r1[3] = abs(pr1 - (pstd1 * norm.ppf(1 - alpha)))

# PORTFOLIO BETA => sum(weight * beta)
for j in range(len(twl1)):
    pb1 += twl1[j] * sb1[j]
r1[4] = pb1

# TREYNOR RATIO => (portfolio_return - risk_free_rate) / portfolio_beta
r1[5] = (r1[0] - risk_free_rate) / r1[4]

# JENSEN ALPHA => portfolio_return - (risk_free_rate + portfolio_beta * (market_return - risk_free_rate))
r1[6] = r1[0] - (risk_free_rate + r1[4] * (market_return - risk_free_rate))   

r1 = pd.Series(r1, index=metrics)
metrics_df = metrics_df.append(r1, ignore_index=True)

In [26]:
# P2

r2 = np.zeros(len(metrics))
sb2 = monthly_stock_beta[portfolio2].copy()
pb2 = 0
tdf2 = monthly_return[portfolio1].copy()
twl2 = w2
tcovdf2 = tdf2.cov()

pr2 = np.sum(tdf2.mean() * twl2) * days
pstd2 = np.sqrt(np.dot(twl2.T,np.dot(tcovdf2, twl2))) * np.sqrt(days)

r2[0] = pr2
r2[1] = pstd2

# SHARPE RATIO => (portfolio_return - risk_free_rate) / volatility
r2[2] = (r2[0] - risk_free_rate) / r2[1]

# VaR => return - (Z * volatility)
r2[3] = abs(pr2 - (pstd2 * norm.ppf(1 - alpha)))

# PORTFOLIO BETA => sum(weight * beta)
for j in range(len(twl2)):
    pb2 += twl2[j] * sb2[j]
r2[4] = pb2

# TREYNOR RATIO => (portfolio_return - risk_free_rate) / portfolio_beta
r2[5] = (r2[0] - risk_free_rate) / r2[4]

# JENSEN ALPHA => portfolio_return - (risk_free_rate + portfolio_beta * (market_return - risk_free_rate))
r2[6] = r2[0] - (risk_free_rate + r2[4] * (market_return - risk_free_rate))   

r2 = pd.Series(r2, index=metrics)
metrics_df = metrics_df.append(r2, ignore_index=True)

In [27]:
metrics_df

,Return,Volatility,Sharpe Ratio,VaR,Portfolio Beta,Treynor Ratio,Jensen Alpha
0,0.817713,0.017712,42.767485,0.776508,1.52831,0.495654,0.818294
1,0.671767,0.026595,22.995904,0.609899,2.16095,0.283008,0.697509
